<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/ReinforcementLearning/RL-ASSIGNMENT2/CH24M571_AASIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Soumya Mukherjee - ch24m571

latest version of the code will be at https://github.com/aymuos/masters-practise-repo/blob/main/TERM3/ReinforcementLearning/RL-ASSIGNMENT2/CH24M571_AASIGNMENT.ipynb

In [26]:
#!pip install beautifulsoup4 requests


I have imported all words into a text as words.txt from the website in the given link

In [27]:
# imports
from collections import defaultdict, Counter
from typing import List, Tuple, Dict
import requests
from bs4 import BeautifulSoup
import numpy as np
import random
from math import log2

In [28]:
def load_words(file_path: str = "words.txt") -> List[str]:
    '''
    This will return back a list of strings read from a local file,
    filtered by length (4 letters) and with duplicates removed.
    '''
    words = []
    try:

      with open("words.txt", 'r') as f:
        for line in f:
            for word in line.split():
                if len(word) == 4 and len(set(word)) == 4: # Filter for 4-letter unique-letter words
                    words.append(word)

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return []

    return list(set(words))

In [29]:
# Feedback function
def cats_dogs(secret: str, guess: str) -> Tuple[int,int]:
    """Return (cats, dogs) feedback between secret and guess.
       dogs = correct letter in correct position
       cats = correct letter but wrong position
    """
    dogs = sum(s == g for s,g in zip(secret, guess))
    cats = sum((Counter(secret) & Counter(guess)).values()) - dogs
    return cats, dogs

In [30]:

def is_consistent(candidate: str, guess: str, feedback: Tuple[int,int]) -> bool:
    """Check if candidate word is consistent with given guess+feedback"""
    return cats_dogs(candidate, guess) == feedback

# Minimax strategy

def partition_sizes_for_guess(guess: str, candidates: List[str]) -> Dict[Tuple[int,int], int]:
    buckets = defaultdict(int)
    for c in candidates:
        buckets[cats_dogs(c, guess)] += 1
    return buckets

def next_guess_minimax(candidates: List[str]) -> str:
    """Choose the guess that minimizes the worst-case remaining candidates."""
    best_guess, best_score = None, float('inf')
    for g in candidates:
        worst = max(partition_sizes_for_guess(g, candidates).values())
        if worst < best_score:
            best_score, best_guess = worst, g
    return best_guess


# Overlap heuristic (for the later stages)

def likely_fixed_positions(candidates: List[str]):

    """Identify positions that are fixed (all candidates agree on the letter)."""

    fixed = {}
    for i in range(len(candidates[0])):
        letters = {w[i] for w in candidates}
        if len(letters) == 1:
            fixed[i] = next(iter(letters))
    return fixed


def next_guess_overlap(candidates: List[str]) -> str:

    """Pick guess biased by letter frequency and fixed 'dog' positions."""


    letter_counts = Counter("".join(candidates))
    fixed = likely_fixed_positions(candidates)

    def score(word):
        s = sum(letter_counts[ch] for ch in set(word))
        for i, ch in fixed.items():
            if word[i] == ch:
                s += 5  # adding a bonus such that the algo prioritises the words that align with already known correct letters
        return s

    # Ensure candidates is not empty before calling max
    if not candidates:
        return ""
    return max(candidates, key=score)


# Hybrid controller

def hybrid_next_guess(candidates: List[str], step: int, minimax_steps: int = 2, switch_threshold: int = 300) -> str:

    """Hybrid strategy : use minimax in first steps or while candidates are large, then switch."""
    if not candidates:
      return ""     # Return empty string if candidates is empty

    if step <= minimax_steps or len(candidates) > switch_threshold:
        return next_guess_minimax(candidates)
    else:
        return next_guess_overlap(candidates)

# Full play loop

def solve_secret(words: List[str], secret: str, minimax_steps: int = 2, switch_threshold: int = 300):
    """Simulate solving the secret with hybrid strategy."""
    candidates = words[:]
    step = 1
    history = []

    while True:
        guess = hybrid_next_guess(candidates, step, minimax_steps, switch_threshold)
        if not guess:
            print("No possible words left. Could not solve.")
            break

        cats, dogs = cats_dogs(secret, guess)
        history.append((step, guess, cats, dogs, len(candidates)))

        if guess == secret:
            break

        # prune inconsistent candidates - words that cannot be the secret word
        candidates = [c for c in candidates if is_consistent(c, guess, (cats, dogs))]
        step += 1

    return history


In [31]:
# Execute code

secret = "POSY"


words = load_words()

print(f"Loaded {len(words)} valid words")

if len(words) > 0:
    if secret in words:
        result = solve_secret(words, secret)

        for step, guess, cats, dogs, remaining in result:
            print(f"Step {step}: Guess={guess}, Cats={cats}, Dogs={dogs}, Remaining={remaining}")

        print(f"Solved in {len(result)} steps! Secret was {secret}")
    else:
        print(f"Secret word '{secret}' not found in the loaded word list.")
else:
    print("No valid words loaded. Cannot run the game.")

Loaded 4309 valid words
Step 1: Guess=TARS, Cats=1, Dogs=0, Remaining=4309
Step 2: Guess=POET, Cats=0, Dogs=2, Remaining=1017
Step 3: Guess=LOUT, Cats=0, Dogs=1, Remaining=49
Step 4: Guess=HOER, Cats=0, Dogs=1, Remaining=13
Step 5: Guess=YBET, Cats=1, Dogs=0, Remaining=3
Step 6: Guess=POSY, Cats=0, Dogs=4, Remaining=1
Solved in 6 steps! Secret was POSY


In [ ]:
# Load all words
words = load_words()

if len(words) > 0:
    total_guesses = 0
    num_words_tested = 0
    results = {}

    print(f"Testing game performance on all {len(words)} words...")

    for secret_word in words:
        if secret_word in words:
            result = solve_secret(words, secret_word)
            guesses_for_word = len(result)
            total_guesses += guesses_for_word
            num_words_tested += 1
            results[secret_word] = guesses_for_word

    if num_words_tested > 0:
        average_guesses = total_guesses / num_words_tested
        print(f"\n--- Overall Performance ---")
        print(f"Tested on {num_words_tested} words.")
        print(f"Average number of guesses: {average_guesses:.2f}")

        if results:
            toughest_word = max(results, key=results.get)
            easiest_word = min(results, key=results.get)
            print(f"Word that took most guesses: {toughest_word} ({results[toughest_word]} guesses)")
            print(f"Word that took least guesses: {easiest_word} ({results[easiest_word]} guesses)")

    else:
        print("No words were tested. Please ensure words are loaded correctly.")
else:
    print("No valid words loaded. Cannot test performance.")

Testing game performance on all 4309 words...
